In [2]:
import numpy as np
import pyedflib
import statistics
import plotly.graph_objects as go
import pandas as pd
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, PersistenceImage 
from gtda.plotting import plot_point_cloud, plot_heatmap, plot_diagram
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA, FastICA
from numpy.linalg import norm
from scipy.stats import skew, kurtosis

pd.set_option('display.max_columns', None)

# Load Data and set important variables

In [9]:
# Choose if you want to look at EEG or EMG data

data_type = "EEG"
#data_type = "EMG"

In [10]:
# choose individuum
subject = "m292"

In [11]:
label_list = [0, 1, 2, 3, 4]

In [12]:
# Load persistence diagrams

persistence_diagrams  = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/'+str(data_type)+'/Persistence_Diagrams_All_Labels.npy', \
    allow_pickle=True).item() # .item() to convert the dtype to dict again

In [13]:
reshaped_persistence_diagrams = {}

for label in label_list:
    reshaped_persistence_diagrams["Label_"+str(label)] = [persistence_diagram[0] for persistence_diagram in list(persistence_diagrams["Label_"+str(label)])]

persistence_diagrams = reshaped_persistence_diagrams

# Define Function Wrapper for All Statistics

In [14]:
def signature_statistics_wrapper(func): 

    def compute_statistics_for_signature(*args, **kwargs):

        # Get variables
        signatures = kwargs.pop('signatures', None)
        label_list = kwargs.pop('label_list', None)
        persistence_diagrams = kwargs.pop('persistence_diagrams', None)

    
        overall_statistics = {}

            
        for label in label_list:

            # Initialize statistics list for label
            overall_statistics["Label_"+str(label)] = []

            for dim in range(3):

                statistics_for_dim = []

                # Take average of the statistics of 25 persistence diagrams
                for idx in range(int(len(persistence_diagrams["Label_"+str(label)]))):
                    # Initialize list for the statistics of each of the the 80 PD after idx

                    sgn = signatures[label][idx].astype("float")

                    statistics_for_dim.append(func(sgn, dim))


                overall_statistics["Label_"+str(label)].append(statistics_for_dim)
                    

        return overall_statistics

    return compute_statistics_for_signature

In [15]:
def precompute_signature(SG, label_list = label_list, persistence_diagrams = persistence_diagrams):
    
    signatures = {}
    for label in label_list:
        signatures[label] = []
        
        for idx in range(int(len(persistence_diagrams["Label_"+str(label)]))):
            
            signatures[label].append(SG.fit_transform([persistence_diagrams["Label_"+str(label)][idx].astype("float")]))

    return signatures

# HeatKernel Statistics

In a way, the Heat Kernel shows an "average distribution" of the persistence diagrams for each label, seperated per hole dimensionality.

In [16]:
HK = HeatKernel(sigma=0.00003, n_bins=100)

heatkernels = precompute_signature(HK)

In [17]:
heat_kernel_statistics = {}

## Intensity

In [18]:
@signature_statistics_wrapper
def heat_kernel_intensity(heatkernel, homology_dimension):
    """ Computes mean intensity of a heatkernel. Only takes positive values because otherwise the mean would
      always be zero.

    Parameters:
    - heatkernel (list of lists): heatkernel of all homology dimensions
    - homology_dimension (int): Which homology dimension to look at (0, 1 or 2)

    Returns:
    - mean intensity of heatkernel of homology dimension homology_dimension
    """
    
    positives =  [x for inner_list in heatkernel[0][homology_dimension] for x in inner_list if x > 0]
    
    return np.mean(positives) if len(positives) > 0 else 0

In [19]:
heat_kernel_statistics["intensity"] = heat_kernel_intensity(signatures = heatkernels, label_list = label_list, persistence_diagrams = persistence_diagrams, heatkernel = None, homology_dimension = None)

## Maximum and Minimum

In [20]:
@signature_statistics_wrapper
def heat_kernel_max(heatkernel, homology_dimension):
    """ Computes maximum and minimum of a heatkernel. 

    Parameters:
    - heatkernel (list of lists): heatkernel of all homology dimensions
    - homology_dimension (int): Which homology dimension to look at (0, 1 or 2)

    Returns:
    - mean intensity of heatkernel of homology dimension homology_dimension
    """
    
    positives =  [x for inner_list in heatkernel[0][homology_dimension] for x in inner_list if x > 0]
    
    return np.max(positives) if len(positives) > 0 else 0

In [21]:
@signature_statistics_wrapper
def heat_kernel_min(heatkernel, homology_dimension):
    """ Computes maximum and minimum of a heatkernel. Only takes positive values because otherwise the minimum
    would always be the negative of the maximum

    Parameters:
    - heatkernel (list of lists): heatkernel of all homology dimensions
    - homology_dimension (int): Which homology dimension to look at (0, 1 or 2)

    Returns:
    - mean intensity of heatkernel of homology dimension homology_dimension
    """
    
    positives =  [x for inner_list in heatkernel[0][homology_dimension] for x in inner_list if x > 0]
    
    return np.min(positives) if len(positives) > 0 else 0

In [22]:
heat_kernel_statistics["maximum"] = heat_kernel_max(signatures = heatkernels, label_list = label_list, persistence_diagrams = persistence_diagrams, heatkernel = None, homology_dimension = None)
heat_kernel_statistics["minimum"] = heat_kernel_min(signatures = heatkernels, label_list = label_list, persistence_diagrams = persistence_diagrams, heatkernel = None, homology_dimension = None)

# Persistence Image Statistics

In [23]:
PI = PersistenceImage(sigma=0.00003, n_bins=100)

images = precompute_signature(PI)

In [24]:
persistence_image_statistics = {}

In [25]:
persistence_image_statistics["intensity"] = heat_kernel_intensity(signatures = images, label_list = label_list, persistence_diagrams = persistence_diagrams, heatkernel = None, homology_dimension = None)
persistence_image_statistics["maximum"] = heat_kernel_max(signatures = images, label_list = label_list, persistence_diagrams = persistence_diagrams, heatkernel = None, homology_dimension = None)
persistence_image_statistics["minimum"] = heat_kernel_min(signatures = images, label_list = label_list, persistence_diagrams = persistence_diagrams, heatkernel = None, homology_dimension = None)

# Betti Curve Features

In [26]:
BC = BettiCurve()

betti_curves = precompute_signature(BC)

In [27]:
betti_curve_statistics = {}

## L1 Norm

In [28]:
@signature_statistics_wrapper
def L1_norm(signature, homology_dimension):

    return norm(signature[0][0], homology_dimension)    

In [29]:
betti_curve_statistics["L1"] = L1_norm(signatures = betti_curves, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)

## Mean, Standard deviation, Skewness and Kurtosis

In [30]:
@signature_statistics_wrapper
def signature_mean(signature, homology_dimension):
    
    return statistics.mean(signature[0][homology_dimension])

In [31]:
@signature_statistics_wrapper
def signature_standard_deviation(signature, homology_dimension):
    
    return statistics.stdev(signature[0][homology_dimension])


In [32]:
@signature_statistics_wrapper
def signature_skewness(signature, homology_dimension):
    
    return skew(signature[0][homology_dimension])

In [33]:
@signature_statistics_wrapper
def signature_kurtosis(signature, homology_dimension):
    
    return kurtosis(signature[0][homology_dimension])

In [34]:
betti_curve_statistics["Mean"] = signature_mean(signatures = betti_curves, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
betti_curve_statistics["Standard_Deviation"] = signature_standard_deviation(signatures = betti_curves, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
betti_curve_statistics["Skewness"] = signature_skewness(signatures = betti_curves, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
betti_curve_statistics["Kurtosis"] = signature_kurtosis(signatures = betti_curves, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)

# Persistence Landscape Features

In [35]:
PL = PersistenceLandscape()
landscapes = precompute_signature(PL)

landscape_statistics = {}

In [36]:
landscape_statistics["Mean"] = signature_mean(signatures = landscapes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
landscape_statistics["Standard_Deviation"] = signature_standard_deviation(signatures = landscapes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
landscape_statistics["Skewness"] = signature_skewness(signatures = landscapes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
landscape_statistics["Kurtosis"] = signature_kurtosis(signatures = landscapes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)

# Silhouette Features

In [37]:
SH = Silhouette()
silhouettes = precompute_signature(SH)

silhouette_statistics = {}

L1 norm, mean, SD, Skewness, Kurtosis

In [38]:
silhouette_statistics["L1"] = L1_norm(signatures = silhouettes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)

silhouette_statistics["Mean"] = signature_mean(signatures = silhouettes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
silhouette_statistics["Standard_Deviation"] = signature_standard_deviation(signatures = silhouettes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
silhouette_statistics["Skewness"] = signature_skewness(signatures = silhouettes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)
silhouette_statistics["Kurtosis"] = signature_kurtosis(signatures = silhouettes, label_list = label_list, persistence_diagrams = persistence_diagrams, signature = None, homology_dimension = None)

# Entries of vectorizations as direct features

In [39]:
def compute_vectorizations(persistence_diagrams, label_list, signatures):
    
    vectorizations = {}

    for label in label_list:
        
        vectorizations["Label_"+str(label)] = []
    
        for idx in range(len(persistence_diagrams["Label_"+str(label)])):
            
            signature = signatures[label][idx]

            vectorizations["Label_"+str(label)].append(signature)

    return vectorizations



In [40]:
def reshape_vectorizations_type_1(vectorizations, label_list):
    """
    Reshape vectorizations of shape (num_persistence_diagrams, 1, 3, 100) to separate dimensions for each label.

    Parameters:
    - vectorizations (dict): Dictionary containing vectorizations for each label.
    - label_list (list): List of labels.

    Returns:
    - reshaped_vectorizations (dict): Dictionary containing reshaped vectorizations for each label and dimension.
    """
    # Initialize dictionary to store reshaped vectorizations
    reshaped_vectorizations = {}
    
    # Iterate over each label
    for label in label_list:
        # Initialize dictionary to store reshaped vectorizations for the current label
        reshaped_vectorizations["Label_" + str(label)] = {}

        for coordinate_idx in range(100):

            # For each vectorization coordinate (there are 100), initialize one dictionary
            # which will contain lists of 74 vectorization coordinates (the "vectorization_idx"st coordinate
            # of the 74 persistence images) as values and the homology dimensions as keys
            reshaped_vectorizations["Label_" + str(label)]["Coordinate_" + str(coordinate_idx)] = {}
 
            for hom_dim in range(3):
                # Initialize list to store reshaped vectorizations for the current homology dimension
                reshaped_vectorizations["Label_" + str(label)]["Coordinate_" + str(coordinate_idx)]["Hom_Dim_" + str(hom_dim)] = []

    
    # Iterate over each label
    for label in label_list:
        # Iterate over each vectorization for the current label
        for coordinate_idx in range(100):
            
            for hom_dim in range(3):
                
                for vectorization_idx in range(len(vectorizations["Label_" + str(label)])):

                    # Append the component corresponding to the current homology dimension to the list
                    reshaped_vectorizations["Label_" + str(label)]["Coordinate_" + str(coordinate_idx)]["Hom_Dim_" + str(hom_dim)].append(
                        vectorizations["Label_" + str(label)][vectorization_idx][0][hom_dim][coordinate_idx])

    return reshaped_vectorizations


In [41]:
# Initialize vectorizations
vectorizations = {}

## Persistence Landscape

First precompute.

In [42]:
PL = PersistenceLandscape()

landscapes = precompute_signature(PL)

In [43]:
vectorizations_before_reshaping = compute_vectorizations(persistence_diagrams, label_list, landscapes)
vectorizations["PL"] = reshape_vectorizations_type_1(vectorizations_before_reshaping, label_list)

## Betti Curve

In [44]:
vectorizations_before_reshaping = compute_vectorizations(persistence_diagrams, label_list, betti_curves)
vectorizations["BC"] = reshape_vectorizations_type_1(vectorizations_before_reshaping, label_list)

## Silhouette

In [45]:
vectorizations_before_reshaping = compute_vectorizations(persistence_diagrams, label_list, silhouettes)
vectorizations["SH"] = reshape_vectorizations_type_1(vectorizations_before_reshaping, label_list)

# Save Signature Features

In [46]:
def create_feature_df(data_type, heat_kernel_statistics, betti_curve_statistics, silhouette_statistics, persistence_image_statistics, landscape_statistics, vectorizations, num_diagrams, label):
    """
    Create DataFrame for each label from features

    Parameters:
    - kernel_densities (list): intensities of heatkernel
    - L1_norms (list): L1 norms of signatures
    - num_diagrams (int): How many diagrams are there in total?
    - label (int): Label for which we want to create a dataframe.

    Returns:
    - Feature DataFrame (DataFrame)
    """
    
    feature_df = pd.DataFrame(index=np.arange(0, num_diagrams))

    for stat in heat_kernel_statistics.keys():
        for homology_dim in range(3):
            feature_df[str(data_type)+"_HeatKernel_Statistic_"+str(stat)+"Dim"+str(homology_dim)] = heat_kernel_statistics[stat]["Label_"+str(label)][homology_dim]

    for stat in betti_curve_statistics.keys():
        for homology_dim in range(3):
            feature_df[str(data_type)+"_Betti_Curve_Statistic_"+str(stat)+"Dim"+str(homology_dim)] = betti_curve_statistics[stat]["Label_"+str(label)][homology_dim]

    for stat in silhouette_statistics.keys():
        for homology_dim in range(3):
            feature_df[str(data_type)+"_Silhouette_Statistic_"+str(stat)+"Dim"+str(homology_dim)] = silhouette_statistics[stat]["Label_"+str(label)][homology_dim]


    for stat in persistence_image_statistics.keys():
        for homology_dim in range(3):
            feature_df[str(data_type)+"_Persistence_image_Statistic_"+str(stat)+"Dim"+str(homology_dim)] = persistence_image_statistics[stat]["Label_"+str(label)][homology_dim]


    for stat in landscape_statistics.keys():
        for homology_dim in range(3):
            feature_df[str(data_type)+"_Persistence_Landscape_Statistic_"+str(stat)+"Dim"+str(homology_dim)] = landscape_statistics[stat]["Label_"+str(label)][homology_dim]

    # Vectorizations
    for signature in vectorizations.keys():
        for homology_dim in range(3):
            for coordinate_idx in range(100):
                feature_df[str(data_type)+"_"+str(signature)+"_Vectorization_Coordinate_"+str(coordinate_idx)+\
                "_Homology_Dim_"+str(homology_dim)] = vectorizations[signature]["Label_"+str(label)]["Coordinate_" + \
                str(coordinate_idx)]["Hom_Dim_" + str(homology_dim)]

    # Label
    feature_df["Label"] = label

    return feature_df

In [47]:
dataframes = {}

for label in label_list:
    dataframes["Label_"+str(label)] = create_feature_df(data_type, heat_kernel_statistics, betti_curve_statistics, silhouette_statistics, persistence_image_statistics, landscape_statistics, vectorizations, 74, label)

/var/folders/f_/tyg450s17m53dh3ylrjcmpqc0000gn/T/ipykernel_37929/329001922.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[str(data_type)+"_"+str(signature)+"_Vectorization_Coordinate_"+str(coordinate_idx)+\
/var/folders/f_/tyg450s17m53dh3ylrjcmpqc0000gn/T/ipykernel_37929/329001922.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[str(data_type)+"_"+str(signature)+"_Vectorization_Coordinate_"+str(coordinate_idx)+\
/var/folders/f_/tyg450s17m53dh3ylrjcmpqc0000gn/T/ipykernel_37929/329001922.py:43: Perf

In [49]:
# Concatenate and save features of training persistence diagrams
feature_df = pd.concat([dataframes["Label_"+str(0)], dataframes["Label_"+str(1)], dataframes["Label_"+str(2)], 
                        dataframes["Label_"+str(3)], dataframes["Label_"+str(4)]], ignore_index=True)
feature_df.to_csv("Features/"+str(subject)+"/"+str(data_type)+"/Signature_Statistics.csv")